## Input Parameters

In [1]:
# The default is to include points from all NLCD classes and MGRS grid zones
point_nlcd_keep_list = []
point_mgrs_keep_list = []

# # This would show all NLCD classes for MGRS grid zone 12T
# point_nlcd_keep_list = [31, 41, 42, 43, 52, 71, 81, 82, 90, 95]
# point_mgrs_keep_list = ['12T']

# # This would show NLCD 82 points for all MGRS grid zones
# point_nlcd_keep_list = [82]
# point_mgrs_keep_list = []

# This would show NLCD 82 points for all MGRS grid zones
point_nlcd_keep_list = [82]
point_mgrs_keep_list = ['10S']

# The minimum number of months in the target year
min_month_count = 3

# Overwrite the summary stats file everytime this block is run
output_txt = 'summary_stats_test.txt'
output_f = open(output_txt, 'w')
output_f.write('MGRS: '+ ', '.join(point_mgrs_keep_list) + '\n')
output_f.write('NLCD: '+ ', '.join(map(str, point_nlcd_keep_list)) + '\n\n')
output_f.close()

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import math
import os
import pprint
import random

#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'conor',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)
    
    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)

        # Conor's Approach
        # There is probably an easier way, but splitting the dataframe at the target index seemed to work pretty well
        window_prev_df = window_df.iloc[:tgt_i]
        window_next_df = window_df.iloc[tgt_i+1:]
        prev_index = window_prev_df['etf'].last_valid_index()
        next_index = window_next_df['etf'].first_valid_index()
        prev_i = window_df.index.get_loc(prev_index)
        next_i = window_df.index.get_loc(next_index)
        w_prev = 0.5 * math.exp(1 - (tgt_i - prev_i))
        w_next = 0.5 * math.exp(1 - (next_i - tgt_i))
        value_prev = window_df['etf'].iloc[prev_i]
        value_next = window_df['etf'].iloc[next_i]
        climo_prev = month_climo_dict[point_id].loc[prev_index.month, 'etf']
        climo_next = month_climo_dict[point_id].loc[next_index.month, 'etf']
        conor = w_prev * (value_prev - climo_prev) + w_next * (value_next - climo_next) + climo_mean

        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'conor': conor,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
            
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[], title='', print_flag=True, write_flag=True):
    """"""
    output = [title]

    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    output.append('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])

        output.append('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))

    if print_flag:
        print('\n'.join(output))
    if write_flag:
        with open(output_txt, 'a') as output_f:
            output_f.write('\n'.join(output+ ['\n']))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 24


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop one datapoint in each year',
    print_flag=True, write_flag=True
)

Randomly drop one datapoint in each year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1241   0.0937   0.0063   0.7732   0.1352   0.7802      192
        climo_mean   0.1613   0.1243  -0.0002   0.6222   0.2145   0.6273      192
      climo_median   0.1661   0.1174  -0.0019   0.7003   0.1684   0.6163      192
             conor   0.1065   0.0836   0.0017   0.8522   0.0858   0.8377      192
     interp_clim_a   0.1232   0.0975   0.0031   0.6977   0.1749   0.7944      192
     interp_clim_b   0.1232   0.0975   0.0033   0.6981   0.1748   0.7944      192
     interp_clim_c   0.1224   0.0929   0.0022   0.7368   0.1518   0.7891      192
       whit_a_0p50   0.1277   0.0966   0.0065   0.7676   0.1385   0.7672      192
       whit_a_0p20   0.1229   0.0924   0.0059   0.8035   0.1175   0.7846      192
       whit_a_0p10   0.1218   0.0913   0.0057   0.8229   0.1064   0.7894      192
       whit_a_0p05   0.1220   0.0912   0.0057   0.8357   

## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "growing" season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "growing" season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1050   0.0769  -0.0010   0.8578   0.0809   0.8770      192
        climo_mean   0.1712   0.1277  -0.0026   0.6553   0.1959   0.6724      192
      climo_median   0.1805   0.1184   0.0011   0.7232   0.1605   0.6448      192
             conor   0.0905   0.0663   0.0025   0.9084   0.0552   0.9084      192
     interp_clim_a   0.1184   0.0930  -0.0018   0.7565   0.1384   0.8543      192
     interp_clim_b   0.1184   0.0930  -0.0017   0.7567   0.1383   0.8543      192
     interp_clim_c   0.1209   0.0896   0.0000   0.7905   0.1207   0.8392      192
       whit_a_0p50   0.1120   0.0821  -0.0003   0.8664   0.0766   0.8597      192
       whit_a_0p20   0.1064   0.0785  -0.0003   0.8957   0.0597   0.8737      192
       whit_a_0p10   0.1042   0.0773  -0.0006   0.9110   0.0507   0.8796      192
       whit_a_0p05   0.1033 

## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1420   0.1115   0.0123   0.7248   0.1712   0.6716      192
        climo_mean   0.1526   0.1180   0.0042   0.6041   0.2329   0.6136      192
      climo_median   0.1555   0.1119   0.0074   0.6544   0.2070   0.6033      192
             conor   0.1242   0.0963   0.0046   0.8155   0.1111   0.7496      192
     interp_clim_a   0.1296   0.1020   0.0083   0.6644   0.2020   0.7275      192
     interp_clim_b   0.1296   0.1020   0.0081   0.6647   0.2017   0.7273      192
     interp_clim_c   0.1296   0.1004   0.0098   0.6896   0.1891   0.7242      192
       whit_a_0p50   0.1474   0.1155   0.0161   0.7233   0.1759   0.6512      192
       whit_a_0p20   0.1442   0.1128   0.0141   0.7510   0.1579   0.6682      192
       whit_a_0p10   0.1431   0.1110   0.0126   0.7674   0.1469   0.6751      192
       whit_a_0p05   0.14

## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a datapoint from the winter (Dec-Feb)',
    print_flag=True, write_flag=True
)

Randomly drop a datapoint from the winter (Dec-Feb)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1405   0.1068  -0.0110   0.6951   0.1804   0.6551      192
        climo_mean   0.1516   0.1209   0.0049   0.5459   0.2900   0.5977      192
      climo_median   0.1516   0.1135   0.0070   0.6117   0.2508   0.5946      192
             conor   0.1231   0.0949  -0.0022   0.7351   0.1641   0.7319      192
     interp_clim_a   0.1306   0.1024  -0.0030   0.6205   0.2352   0.7092      192
     interp_clim_b   0.1306   0.1025  -0.0030   0.6209   0.2350   0.7093      192
     interp_clim_c   0.1291   0.0988  -0.0020   0.6534   0.2156   0.7096      192
       whit_a_0p50   0.1461   0.1094  -0.0140   0.6819   0.1858   0.6296      192
       whit_a_0p20   0.1417   0.1082  -0.0074   0.7172   0.1702   0.6519      192
       whit_a_0p10   0.1405   0.1084  -0.0034   0.7370   0.1618   0.6596      192
       whit_a_0p05   0.1407   0.1092  -0.0005 

## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint that is next to an existing missing data point',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint that is next to an existing missing data point
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1265   0.1183  -0.0160   0.7137   0.1731   0.5853       17
        climo_mean   0.1330   0.0925  -0.0448   0.4301   0.3316   0.6133       17
      climo_median   0.1291   0.0884  -0.0436   0.4664   0.3089   0.6237       17
             conor   0.1031   0.0881  -0.0279   0.6175   0.2247   0.7448       17
     interp_clim_a   0.1175   0.1015  -0.0304   0.5719   0.2523   0.6460       17
     interp_clim_b   0.1177   0.1020  -0.0300   0.5762   0.2499   0.6421       17
     interp_clim_c   0.1153   0.0981  -0.0298   0.5900   0.2410   0.6579       17
       whit_a_0p50   0.1381   0.1247  -0.0136   0.7285   0.1657   0.5367       17
       whit_a_0p20   0.1386   0.1259  -0.0167   0.8209   0.1016   0.5736       17
       whit_a_0p10   0.1420   0.1251  -0.0194   0.8667   0.0686   0.5822       17
       whit_a_0p05

## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1533   0.1160  -0.0160   0.7044   0.1586   0.7027      192
        climo_mean   0.1842   0.1383  -0.0020   0.5881   0.2413   0.5672      192
      climo_median   0.1957   0.1329   0.0015   0.6493   0.2087   0.5348      192
             conor   0.1326   0.1013  -0.0042   0.7569   0.1394   0.7761      192
     interp_clim_a   0.1503   0.1182  -0.0090   0.6463   0.1999   0.7197      192
     interp_clim_b   0.1502   0.1182  -0.0090   0.6466   0.1998   0.7199      192
     interp_clim_c   0.1532   0.1156  -0.0073   0.6768   0.1836   0.7013      192
       whit_a_0p50   0.1495   0.1132  -0.0147   0.7443   0.1363   0.7179      192
       whit_a_0p20   0.1502   0.1140  -0.0160   0.7726   0.1183   0.7190      192
       whit_a_0p10   0.1530   0.1154  -0.0169   0.7842   0.1106   0.7118      192
       whit_a_0p05   0.1563   0

## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1640   0.1178  -0.0043   0.6611   0.1888   0.7021      192
        climo_mean   0.1842   0.1352  -0.0132   0.6062   0.2112   0.6233      192
      climo_median   0.1952   0.1286  -0.0090   0.6799   0.1734   0.5893      192
             conor   0.1299   0.0932  -0.0123   0.7402   0.1358   0.8211      192
     interp_clim_a   0.1527   0.1146  -0.0088   0.6337   0.2000   0.7622      192
     interp_clim_b   0.1527   0.1146  -0.0088   0.6337   0.2000   0.7622      192
     interp_clim_c   0.1539   0.1120  -0.0067   0.6705   0.1811   0.7429      192
       whit_a_0p50   0.1703   0.1210  -0.0066   0.6842   0.1734   0.6765      192
       whit_a_0p20   0.1636   0.1169  -0.0083   0.7111   0.1563   0.7021      192
       whit_a_0p10   0.1606   0.1149  -0.0091   0.7250   0.1477   0.7130      192
       whit_a_0p05   0.1592   0

## Randomly drop a two month gap during the non-growing season (Oct-Mar)

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1556   0.1227  -0.0045   0.6544   0.1996   0.6346      192
        climo_mean   0.1592   0.1207   0.0237   0.6041   0.2576   0.6257      192
      climo_median   0.1657   0.1157   0.0286   0.6429   0.2396   0.6001      192
             conor   0.1244   0.0945   0.0192   0.7510   0.1663   0.7722      192
     interp_clim_a   0.1383   0.1090   0.0096   0.6293   0.2286   0.7243      192
     interp_clim_b   0.1384   0.1092   0.0094   0.6294   0.2283   0.7238      192
     interp_clim_c   0.1389   0.1056   0.0120   0.6487   0.2196   0.7168      192
       whit_a_0p50   0.1550   0.1180  -0.0003   0.6727   0.1930   0.6382      192
       whit_a_0p20   0.1552   0.1195  -0.0011   0.7027   0.1746   0.6416      192
       whit_a_0p10   0.1573   0.1210  -0.0016   0.7190   0.1644   0.6365      192
       whit_a_0p05   0.1599 

## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1608   0.1259  -0.0096   0.7018   0.1664   0.6595      192
        climo_mean   0.1634   0.1244  -0.0027   0.6259   0.2180   0.6450      192
      climo_median   0.1698   0.1191  -0.0020   0.6999   0.1751   0.6251      192
             conor   0.1317   0.1003  -0.0089   0.7422   0.1432   0.7712      192
     interp_clim_a   0.1390   0.1112  -0.0062   0.6639   0.1922   0.7540      192
     interp_clim_b   0.1390   0.1112  -0.0062   0.6642   0.1920   0.7540      192
     interp_clim_c   0.1386   0.1079  -0.0058   0.7008   0.1708   0.7474      192
       whit_a_0p50   0.1711   0.1326  -0.0050   0.7538   0.1402   0.6333      192
       whit_a_0p20   0.1736   0.1341  -0.0069   0.7756   0.1255   0.6318      192
       whit_a_0p10   0.1757   0.1350  -0.0088   0.7846   0.1183   0.6285      192
       whit_a_0p05   0.1777   0.1362  -0.0104   0.

## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1651   0.1184   0.0218   0.7119   0.1773   0.6669      192
        climo_mean   0.1685   0.1224   0.0003   0.6517   0.1883   0.6436      192
      climo_median   0.1739   0.1106  -0.0013   0.7540   0.1315   0.6407      192
             conor   0.1307   0.0930   0.0011   0.7625   0.1293   0.7865      192
     interp_clim_a   0.1439   0.1079   0.0110   0.6818   0.1828   0.7474      192
     interp_clim_b   0.1439   0.1080   0.0111   0.6819   0.1828   0.7472      192
     interp_clim_c   0.1425   0.1011   0.0103   0.7329   0.1544   0.7468      192
       whit_a_0p50   0.1756   0.1251   0.0158   0.7602   0.1451   0.6392      192
       whit_a_0p20   0.1744   0.1258   0.0162   0.7867   0.1313   0.6501      192
       whit_a_0p10   0.1739   0.1260   0.0172   0.8025   0.1237   0.6565      192
       whit_a_0p05   0.1739  

## Randomly drop a three month gap during the non-growing season (Oct-Mar)

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1917   0.1429  -0.0268   0.5699   0.2370   0.4523      192
        climo_mean   0.1628   0.1287   0.0082   0.5564   0.2802   0.5766      192
      climo_median   0.1667   0.1222   0.0060   0.6091   0.2457   0.5590      192
             conor   0.1395   0.1082   0.0033   0.6575   0.2133   0.6896      192
     interp_clim_a   0.1576   0.1222  -0.0093   0.5631   0.2586   0.6060      192
     interp_clim_b   0.1577   0.1223  -0.0096   0.5635   0.2581   0.6051      192
     interp_clim_c   0.1583   0.1199  -0.0104   0.5895   0.2414   0.5998      192
       whit_a_0p50   0.1956   0.1442  -0.0186   0.5914   0.2319   0.4422      192
       whit_a_0p20   0.1977   0.1455  -0.0194   0.6176   0.2151   0.4455      192
       whit_a_0p10   0.2008   0.1474  -0.0202   0.6301   0.2066   0.4409      192
       whit_a_0p05   0.204

## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1994   0.1479  -0.0019   0.6039   0.2261   0.4637      192
        climo_mean   0.1840   0.1291  -0.0146   0.5921   0.2202   0.5203      192
      climo_median   0.1999   0.1259  -0.0165   0.6365   0.1927   0.4764      192
             conor   0.1531   0.1094  -0.0175   0.6789   0.1673   0.6636      192
     interp_clim_a   0.1630   0.1221  -0.0083   0.5980   0.2231   0.6147      192
     interp_clim_b   0.1631   0.1222  -0.0082   0.5985   0.2230   0.6145      192
     interp_clim_c   0.1675   0.1211  -0.0092   0.6202   0.2094   0.5941      192
       whit_a_0p50   0.2048   0.1464   0.0011   0.6855   0.1821   0.4789      192
       whit_a_0p20   0.2042   0.1480  -0.0016   0.7144   0.1628   0.4929      192
       whit_a_0p10   0.2058   0.1512  -0.0039   0.7285   0.1524   0.4945      192
       whit_a_0p05   0.2085   0.1545  -0.0060   0.7

## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1702   0.1276   0.0486   0.6587   0.2219   0.6799      192
        climo_mean   0.1765   0.1295   0.0209   0.6222   0.2128   0.6295      192
      climo_median   0.1862   0.1187   0.0215   0.7144   0.1665   0.6066      192
             conor   0.1460   0.1094   0.0133   0.7062   0.1624   0.7472      192
     interp_clim_a   0.1521   0.1191   0.0348   0.6404   0.2174   0.7523      192
     interp_clim_b   0.1522   0.1191   0.0348   0.6404   0.2174   0.7519      192
     interp_clim_c   0.1509   0.1131   0.0350   0.6865   0.1942   0.7445      192
       whit_a_0p50   0.1728   0.1247   0.0294   0.7336   0.1647   0.6586      192
       whit_a_0p20   0.1758   0.1308   0.0265   0.7666   0.1451   0.6548      192
       whit_a_0p10   0.1795   0.1359   0.0261   0.7833   0.1362   0.6478      192
       whit_a_0p05   0.1834   

## Randomly drop a four month gap during the non-growing season (Oct-Mar)

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2186   0.1611  -0.0561   0.5207   0.2473   0.3592      192
        climo_mean   0.1448   0.1120  -0.0027   0.6154   0.2408   0.6695      192
      climo_median   0.1493   0.1042  -0.0014   0.6657   0.2103   0.6443      192
             conor   0.1346   0.1042  -0.0058   0.6684   0.2042   0.7138      192
     interp_clim_a   0.1585   0.1195  -0.0294   0.5680   0.2441   0.6160      192
     interp_clim_b   0.1590   0.1199  -0.0297   0.5670   0.2445   0.6134      192
     interp_clim_c   0.1575   0.1168  -0.0288   0.5932   0.2288   0.6175      192
       whit_a_0p50   0.2340   0.1655  -0.0477   0.5297   0.2501   0.3123      192
       whit_a_0p20   0.2361   0.1667  -0.0493   0.5547   0.2327   0.3200      192
       whit_a_0p10   0.2373   0.1686  -0.0506   0.5727   0.2200   0.3265      192
       whit_a_0p05   0.2381

## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1984   0.1559   0.0270   0.5644   0.2671   0.5112      192
        climo_mean   0.1608   0.1202   0.0152   0.6494   0.2084   0.6727      192
      climo_median   0.1651   0.1095   0.0160   0.7200   0.1703   0.6599      192
             conor   0.1397   0.1037   0.0153   0.7073   0.1766   0.7564      192
     interp_clim_a   0.1520   0.1196   0.0211   0.6069   0.2378   0.7312      192
     interp_clim_b   0.1521   0.1197   0.0210   0.6071   0.2375   0.7302      192
     interp_clim_c   0.1500   0.1165   0.0215   0.6422   0.2187   0.7282      192
       whit_a_0p50   0.1982   0.1494   0.0195   0.6567   0.2087   0.5316      192
       whit_a_0p20   0.2075   0.1540   0.0190   0.6832   0.1936   0.5115      192
       whit_a_0p10   0.2161   0.1603   0.0189   0.6915   0.1889   0.4897      192
       whit_a_0p05   0.2227   0.1653   0.0192   0.69

## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2160   0.1666   0.0752   0.5628   0.3058   0.4757      192
        climo_mean   0.1638   0.1224  -0.0083   0.6246   0.1897   0.6481      192
      climo_median   0.1701   0.1150  -0.0079   0.7041   0.1482   0.6288      192
             conor   0.1412   0.1059  -0.0059   0.6816   0.1620   0.7429      192
     interp_clim_a   0.1594   0.1243   0.0335   0.5937   0.2477   0.6949      192
     interp_clim_b   0.1599   0.1246   0.0334   0.5929   0.2482   0.6922      192
     interp_clim_c   0.1581   0.1215   0.0337   0.6335   0.2270   0.6903      192
       whit_a_0p50   0.2286   0.1649   0.0522   0.6443   0.2398   0.4410      192
       whit_a_0p20   0.2323   0.1642   0.0446   0.6742   0.2165   0.4401      192
       whit_a_0p10   0.2371   0.1677   0.0415   0.6862   0.2070   0.4319      192
       whit_a_0p05   0.2421   0

## Randomly drop a six month gap during the non-growing season (Oct-Mar)

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2234   0.1692  -0.0583   0.5528   0.2261   0.3605      192
        climo_mean   0.1633   0.1229  -0.0138   0.6118   0.2331   0.5796      192
      climo_median   0.1685   0.1166  -0.0106   0.6801   0.1928   0.5701      192
             conor   0.1472   0.1108  -0.0077   0.6608   0.2080   0.6556      192
     interp_clim_a   0.1617   0.1246  -0.0360   0.5823   0.2296   0.6047      192
     interp_clim_b   0.1619   0.1248  -0.0360   0.5831   0.2291   0.6035      192
     interp_clim_c   0.1602   0.1207  -0.0345   0.6165   0.2094   0.6096      192
       whit_a_0p50   0.2350   0.1762  -0.0355   0.5983   0.2200   0.3386      192
       whit_a_0p20   0.2380   0.1779  -0.0258   0.6217   0.2147   0.3403      192
       whit_a_0p10   0.2400   0.1802  -0.0212   0.6297   0.2143   0.3387      192
       whit_a_0p05   0.2419 

### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )